In [1]:
import pandas as pd
import urllib
import requests
from bs4 import BeautifulSoup as bs
import json

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [3]:
table2=pd.read_csv('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&refine.state=TX&timezone=America/Denver', sep=';')
table2=table2.drop(['timezone', 'dst', 'geopoint'], axis=1)
table2.head()



,zip,city,state,latitude,longitude
0,79110,Amarillo,TX,35.157403,-101.86114
1,75447,Ivanhoe,TX,33.774197,-96.12045
2,77665,Winnie,TX,29.820542,-94.39965
3,78009,Castroville,TX,29.356455,-98.88062
4,78039,La Coste,TX,29.312661,-98.81591


In [4]:
austin_data2=table2[table2['city']=='Austin'].reset_index(drop=True)
austin_data2.head()
print(austin_data2.shape)

(83, 5)


In [5]:
austin_data2.rename({'zip': 'Zipcode', 'city': 'City'}, axis=1, inplace=True)
austin_data2.head()

,Zipcode,City,state,latitude,longitude
0,78749,Austin,TX,30.216108,-97.858280
1,78785,Austin,TX,30.326374,-97.771258
2,78735,Austin,TX,30.250761,-97.844690
3,78761,Austin,TX,30.326374,-97.771258
4,78972,Austin,TX,30.315360,-97.663293


In [6]:
url = 'http://polygons.openstreetmap.fr/get_geojson.py?id=113314&parama=0'
austin_borders  = requests.get(url).json()

address = 'Austin, Texas'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Austin are {}, {}.'.format(latitude, longitude))

C:\Users\owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Austin are 30.2711286, -97.7436995.


In [7]:
map_austin = folium.Map(location=[latitude, longitude], zoom_start=10, tiles = "OpenStreetMap")

# add markers to map
for lat, lng, zipcode in zip(austin_data2['latitude'], austin_data2['longitude'], austin_data2['Zipcode']):
    label = '{},'.format(zipcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_austin)  
    
map_austin.choropleth(geo_data = austin_borders, fill_color=None)

map_austin

In [8]:
# @hidden_cell

CLIENT_ID = 'UISM5KU1JZVMGVGO3MDL1X3XAKO1VMNB4EEQMYVH4CHNMJKT'
CLIENT_SECRET = 'ULT5ZGDXE2YDQIJKO1UT54IWGYGPWLXNBH1I2DL5A1EHYNAN'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UISM5KU1JZVMGVGO3MDL1X3XAKO1VMNB4EEQMYVH4CHNMJKT
CLIENT_SECRET:ULT5ZGDXE2YDQIJKO1UT54IWGYGPWLXNBH1I2DL5A1EHYNAN


In [9]:
# @hidden_cell 
    
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500,):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll=,{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll=30.2711286,-97.743699,&query=hookah'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
rez=requests.get(url2).json()
rez

{'meta': {'code': 200, 'requestId': '5d10fa8ce7065500254e9551'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'suggestedRadius': 2682,
  'headerLocation': 'Downtown Austin',
  'headerFullLocation': 'Downtown Austin, Austin',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hookah',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 31.153098881782494,
    'lng': -97.59791033433737},
   'sw': {'lat': 29.442777482567696, 'lng': -98.75878595925278}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5515eda7498ed8db3d8eccdd',
       'name': 'Rainey Hookah Lounge & Bar',
       'location': {'address': '84 N Interstate 35',
        'lat': 30.25970602164228,
        'lng': -97.73768514116837,

In [11]:
import json
import csv

In [12]:
json.dumps(rez)

'{"meta": {"code": 200, "requestId": "5d10fa8ce7065500254e9551"}, "response": {"suggestedFilters": {"header": "Tap to show:", "filters": [{"name": "$-$$$$", "key": "price"}]}, "suggestedRadius": 2682, "headerLocation": "Downtown Austin", "headerFullLocation": "Downtown Austin, Austin", "headerLocationGranularity": "neighborhood", "query": "hookah", "totalResults": 20, "suggestedBounds": {"ne": {"lat": 31.153098881782494, "lng": -97.59791033433737}, "sw": {"lat": 29.442777482567696, "lng": -98.75878595925278}}, "groups": [{"type": "Recommended Places", "name": "recommended", "items": [{"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "5515eda7498ed8db3d8eccdd", "name": "Rainey Hookah Lounge & Bar", "location": {"address": "84 N Interstate 35", "lat": 30.25970602164228, "lng": -97.73768514116837, "labeledLatLngs": [{"label": "display", "lat": 30.25970602164228, "lng": -97.73768514116837}],

In [13]:
raw= """{"meta": {"code": 200, "requestId": "5d103097f129b500258fb84f"}, "response": {"suggestedFilters": {"header": "Tap to show:", "filters": [{"name": "$-$$$$", "key": "price"}, {"name": "Open now", "key": "openNow"}]}, "suggestedRadius": 2682, "headerLocation": "Downtown Austin", "headerFullLocation": "Downtown Austin, Austin", "headerLocationGranularity": "neighborhood", "query": "hookah", "totalResults": 20, "suggestedBounds": {"ne": {"lat": 31.153098881782494, "lng": -97.59791033433737}, "sw": {"lat": 29.442777482567696, "lng": -98.75878595925278}}, "groups": [{"type": "Recommended Places", "name": "recommended", "items": [{"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "5515eda7498ed8db3d8eccdd", "name": "Rainey Hookah Lounge & Bar", "location": {"address": "84 N Interstate 35", "lat": 30.25970602164228, "lng": -97.73768514116837, "labeledLatLngs": [{"label": "display", "lat": 30.25970602164228, "lng": -97.73768514116837}], "distance": 1396, "postalCode": "78701", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["84 N Interstate 35", "Austin, TX 78701", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "referralId": "e-0-5515eda7498ed8db3d8eccdd-0"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4ad5600df964a520ec0221e3", "name": "Arab Cowboy", "location": {"address": "901 W 24th St", "lat": 30.288071432237427, "lng": -97.7471006957173, "labeledLatLngs": [{"label": "display", "lat": 30.288071432237427, "lng": -97.7471006957173}], "distance": 1914, "postalCode": "78705", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["901 W 24th St", "Austin, TX 78705", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "referralId": "e-0-4ad5600df964a520ec0221e3-1"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4638d082f964a52014461fe3", "name": "Kasbah Moroccan Lounge", "location": {"address": "2714 Guadalupe St", "crossStreet": "27th St", "lat": 30.292657780753558, "lng": -97.74154749691701, "labeledLatLngs": [{"label": "display", "lat": 30.292657780753558, "lng": -97.74154749691701}], "distance": 2405, "postalCode": "78705", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["2714 Guadalupe St (27th St)", "Austin, TX 78705", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "referralId": "e-0-4638d082f964a52014461fe3-2"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "42accc80f964a5204a251fe3", "name": "Karma Lounge", "location": {"address": "105 E 5th St", "crossStreet": "btw Congress and Colorado", "lat": 30.266789240883906, "lng": -97.74226931117228, "labeledLatLngs": [{"label": "display", "lat": 30.266789240883906, "lng": -97.74226931117228}], "distance": 502, "postalCode": "78701", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["105 E 5th St (btw Congress and Colorado)", "Austin, TX 78701", "United States"]}, "categories": [{"id": "4bf58dd8d48988d11f941735", "name": "Nightclub", "pluralName": "Nightclubs", "shortName": "Nightclub", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "referralId": "e-0-42accc80f964a5204a251fe3-3"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4de1d0a0c65b7a3e210cb339", "name": "Off The Hookah Lounge And Smoke Shop", "location": {"address": "2621 Jones Rd Ste A", "lat": 30.224767374060978, "lng": -97.8031957432941, "labeledLatLngs": [{"label": "display", "lat": 30.224767374060978, "lng": -97.8031957432941}], "distance": 7705, "postalCode": "78745", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["2621 Jones Rd Ste A", "Austin, TX 78745", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4de1d0a0c65b7a3e210cb339-4"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4b986cd4f964a520ce4135e3", "name": "Yahala Hookah Lounge", "location": {"address": "6617 Airport Blvd", "lat": 30.331051006247723, "lng": -97.71528036610943, "labeledLatLngs": [{"label": "display", "lat": 30.331051006247723, "lng": -97.71528036610943}], "distance": 7208, "postalCode": "78752", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["6617 Airport Blvd", "Austin, TX 78752", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4b986cd4f964a520ce4135e3-5"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4f321e0f19836c91c7b7abad", "name": "Smoke \'N O\'s Hookah Cafe", "location": {"address": "104 E 31st St", "lat": 30.29515, "lng": -97.73673, "labeledLatLngs": [{"label": "display", "lat": 30.29515, "lng": -97.73673}], "distance": 2756, "postalCode": "78705", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["104 E 31st St", "Austin, TX 78705", "United States"]}, "categories": [{"id": "4d4b7105d754a06374d81259", "name": "Food", "pluralName": "Food", "shortName": "Food", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/food/default_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4f321e0f19836c91c7b7abad-6"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4cb0ea3eef1b3704f42b3f00", "name": "Rawshe Hookah Cafe", "location": {"address": "11331 N Lamar Blvd", "crossStreet": "Lamar and Braker", "lat": 30.382118042039004, "lng": -97.68546794421219, "labeledLatLngs": [{"label": "display", "lat": 30.382118042039004, "lng": -97.68546794421219}], "distance": 13563, "postalCode": "78753", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["11331 N Lamar Blvd (Lamar and Braker)", "Austin, TX 78753", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4cb0ea3eef1b3704f42b3f00-7"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "57b2a83b498e04453164e1ab", "name": "Moonlight Hookah Cafe", "location": {"address": "12636 Research Blvd Ste A109", "crossStreet": "Jollyville Rd.", "lat": 30.430515902564263, "lng": -97.7641636133194, "labeledLatLngs": [{"label": "display", "lat": 30.430515902564263, "lng": -97.7641636133194}], "distance": 17851, "postalCode": "78759", "cc": "US", "city": "Austin", "state": "TX", "country": "United States", "formattedAddress": ["12636 Research Blvd Ste A109 (Jollyville Rd.)", "Austin, TX 78759", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}, "venuePage": {"id": "326741574"}}, "flags": {"outsideRadius": true}, "referralId": "e-0-57b2a83b498e04453164e1ab-8"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4f9ca317e4b0849ad0a0814d", "name": "yazmynes", "location": {"address": "1100 Grand Avenue Pkwy", "lat": 30.458263977098824, "lng": -97.65067740819717, "labeledLatLngs": [{"label": "display", "lat": 30.458263977098824, "lng": -97.65067740819717}], "distance": 22666, "postalCode": "78660", "cc": "US", "city": "Pflugerville", "state": "TX", "country": "United States", "formattedAddress": ["1100 Grand Avenue Pkwy", "Pflugerville, TX 78660", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4f9ca317e4b0849ad0a0814d-9"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "5554fcee498ecc0612a7925a", "name": "Hookah House", "location": {"address": "University Drive", "lat": 29.885485481038167, "lng": -97.93838050701625, "labeledLatLngs": [{"label": "display", "lat": 29.885485481038167, "lng": -97.93838050701625}], "distance": 46846, "cc": "US", "city": "San Marcos", "state": "TX", "country": "United States", "formattedAddress": ["University Drive", "San Marcos, TX", "United States"]}, "categories": [{"id": "4bf58dd8d48988d1ff941735", "name": "Miscellaneous Shop", "pluralName": "Miscellaneous Shops", "shortName": "Shop", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/shops/default_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-5554fcee498ecc0612a7925a-10"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "51dde8bc8bbdabb444345e68", "name": "Vapors Exotic Hookah Lounge and Bar", "location": {"address": "104 W Elms Rd Ste 600", "crossStreet": "Inside Excon gas station plaza", "lat": 31.055121489585783, "lng": -97.65244960784912, "labeledLatLngs": [{"label": "display", "lat": 31.055121489585783, "lng": -97.65244960784912}], "distance": 87709, "postalCode": "76542", "cc": "US", "city": "Killeen", "state": "TX", "country": "United States", "formattedAddress": ["104 W Elms Rd Ste 600 (Inside Excon gas station plaza)", "Killeen, TX 76542", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}, "venuePage": {"id": "76974013"}}, "flags": {"outsideRadius": true}, "referralId": "e-0-51dde8bc8bbdabb444345e68-11"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "57b006d1498ea963d22b7259", "name": "Up In Smoke Hookah Lounge", "location": {"address": "403 E. Stan Schlueter loop suite 304", "lat": 31.075357, "lng": -97.747565, "labeledLatLngs": [{"label": "display", "lat": 31.075357, "lng": -97.747565}], "distance": 89526, "cc": "US", "city": "Killeen", "state": "TX", "country": "United States", "formattedAddress": ["403 E. Stan Schlueter loop suite 304", "Killeen, TX", "United States"]}, "categories": [{"id": "4bf58dd8d48988d121941735", "name": "Lounge", "pluralName": "Lounges", "shortName": "Lounge", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/default_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}, "venuePage": {"id": "500290675"}}, "flags": {"outsideRadius": true}, "referralId": "e-0-57b006d1498ea963d22b7259-12"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4b4d381cf964a52087ce26e3", "name": "Hookah Cafe", "location": {"address": "2250 Thousand Oaks Dr", "crossStreet": "Henderson Pass", "lat": 29.587781517178623, "lng": -98.45277751759922, "labeledLatLngs": [{"label": "display", "lat": 29.587781517178623, "lng": -98.45277751759922}], "distance": 102303, "postalCode": "78232", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["2250 Thousand Oaks Dr (Henderson Pass)", "San Antonio, TX 78232", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4b4d381cf964a52087ce26e3-13"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4b2adc62f964a52010b124e3", "name": "Gypsy Cafe and Hookah Bar", "location": {"address": "2813 Thousand Oaks Dr", "lat": 29.579397290715054, "lng": -98.44272802157299, "labeledLatLngs": [{"label": "display", "lat": 29.579397290715054, "lng": -98.44272802157299}], "distance": 102359, "postalCode": "78232", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["2813 Thousand Oaks Dr", "San Antonio, TX 78232", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4b2adc62f964a52010b124e3-14"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4c81cc7fd92ea093dffb4472", "name": "Studio 13 Hookah Bar", "location": {"address": "7201-7223 Farm-To-Market Road 2696", "crossStreet": "blanco", "lat": 29.52394800857665, "lng": -98.50425460472725, "labeledLatLngs": [{"label": "display", "lat": 29.52394800857665, "lng": -98.50425460472725}], "distance": 110928, "postalCode": "78216", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["7201-7223 Farm-To-Market Road 2696 (blanco)", "San Antonio, TX 78216", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4c81cc7fd92ea093dffb4472-15"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4ba3698ef964a520fb3a38e3", "name": "Naara\'s Cafe", "location": {"address": "9329 Wurzbach Rd", "lat": 29.530318203629374, "lng": -98.56462410202644, "labeledLatLngs": [{"label": "display", "lat": 29.530318203629374, "lng": -98.56462410202644}], "distance": 114351, "postalCode": "78240", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["9329 Wurzbach Rd", "San Antonio, TX 78240", "United States"]}, "categories": [{"id": "4bf58dd8d48988d115941735", "name": "Middle Eastern Restaurant", "pluralName": "Middle Eastern Restaurants", "shortName": "Middle Eastern", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/food/middleeastern_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4ba3698ef964a520fb3a38e3-16"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4bbfe42ef8219c7442e2b010", "name": "Karma Hookah Lounge", "location": {"address": "14531 Runner Rd", "lat": 29.575983698687782, "lng": -98.62585664346997, "labeledLatLngs": [{"label": "display", "lat": 29.575983698687782, "lng": -98.62585664346997}], "distance": 115028, "postalCode": "78249", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["14531 Runner Rd", "San Antonio, TX 78249", "United States"]}, "categories": [{"id": "4bf58dd8d48988d119941735", "name": "Hookah Bar", "pluralName": "Hookah Bars", "shortName": "Hookah Bar", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/hookahbar_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4bbfe42ef8219c7442e2b010-17"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "4c68d8671226c9b6f2d2d9af", "name": "Sultan Hookah Bar", "location": {"lat": 29.5208647486809, "lng": -98.59736651039897, "labeledLatLngs": [{"label": "display", "lat": 29.5208647486809, "lng": -98.59736651039897}], "distance": 117312, "postalCode": "78229", "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["San Antonio, TX 78229", "United States"]}, "categories": [{"id": "4bf58dd8d48988d10f941735", "name": "Indian Restaurant", "pluralName": "Indian Restaurants", "shortName": "Indian", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/food/indian_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-4c68d8671226c9b6f2d2d9af-18"}, {"reasons": {"count": 0, "items": [{"summary": "This spot is popular", "type": "general", "reasonName": "globalInteractionReason"}]}, "venue": {"id": "516dd90ee4b0d88f05a7224e", "name": "Hookah Layal Cafe", "location": {"lat": 29.520519364350186, "lng": -98.70601888539298, "labeledLatLngs": [{"label": "display", "lat": 29.520519364350186, "lng": -98.70601888539298}], "distance": 124924, "cc": "US", "city": "San Antonio", "state": "TX", "country": "United States", "formattedAddress": ["San Antonio, TX", "United States"]}, "categories": [{"id": "4bf58dd8d48988d115941735", "name": "Middle Eastern Restaurant", "pluralName": "Middle Eastern Restaurants", "shortName": "Middle Eastern", "icon": {"prefix": "https://ss3.4sqi.net/img/categories_v2/food/middleeastern_", "suffix": ".png"}, "primary": true}], "photos": {"count": 0, "groups": []}}, "flags": {"outsideRadius": true}, "referralId": "e-0-516dd90ee4b0d88f05a7224e-19"}]}]}}"""

In [18]:
dict_list = []
for item in rez:
    row_dict = {}
    row_dict['column1'] = item['city'][0]['somevalue'] 
    row_dict['column2'] = item['name'][0]['someothervalue']
    
    dict_list.append(row_dict)

TypeError: string indices must be integers

In [ ]:
df = pd.DataFrame(dict_list)

In [15]:
json_normalize(rez_flat)

NameError: name 'rez_flat' is not defined

In [ ]:
austin_venues = getNearbyVenues(names=austin_data2['Zipcode'],
                                   latitudes=austin_data2['latitude'],
                                   longitudes=austin_data2['longitude']
                                  )

In [ ]:
austin_venues.groupby('Zipcode').count()